In [ ]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pymatreader

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
def kinz_data(data):
  numbodies_list = data["kinzData"]['numbodies']
  req_index = []

  for i in range(len(numbodies_list)):
    if numbodies_list[i]==1:
      req_index.append(i)


  kinz_arr = np.empty([0,97])

  for i in req_index:

    timeStamps = np.array(data["kinzData"]['timeStamps'][i])
    kinz_x = data["kinzData"]['bodies'][i]['Position3d'][0]
    kinz_y = data["kinzData"]['bodies'][i]['Position3d'][1]
    kinz_z = data["kinzData"]['bodies'][i]['Position3d'][2]

    d = np.concatenate((timeStamps,data["kinzData"]['bodies'][i]['Position3d'][0], data["kinzData"]['bodies'][i]['Position3d'][1],
                        data["kinzData"]['bodies'][i]['Position3d'][2]), axis=None)


    kinz_arr = np.append(kinz_arr, [d], axis=0)



  arr_walking_speed = np.array([data["walking_speed"]]*len(req_index))
  arr_trial_num = np.array([data["trial_num"]]*len(req_index))
  arr_patientID = np.array([data["patientID"]]*len(req_index))

  #res_kinz_arr = np.column_stack((arr_patientID,arr_trial_num,arr_walking_speed,kinz_arr))
  return np.column_stack((arr_patientID,arr_trial_num,arr_walking_speed,kinz_arr))






def acc_data(data):

  arr_walking_speed = np.array([data["walking_speed"]]*len(data["accelData"]['timeStamps']))
  arr_stopwatch = np.array([data["stopwatch"]]*len(data["accelData"]['timeStamps']))
  arr_trial_num = np.array([data["trial_num"]]*len(data["accelData"]['timeStamps']))
  arr_patientID = np.array([data["patientID"]]*len(data["accelData"]['timeStamps']))

  #res = np.column_stack((arr_patientID,arr_trial_num,arr_walking_speed,arr_stopwatch,data["accelData"]['timeStamps'],data["accelData"]['timeSeries']))
  return np.column_stack((arr_patientID,arr_trial_num,arr_walking_speed,arr_stopwatch,data["accelData"]['timeStamps'],data["accelData"]['timeSeries']))





import numpy as np

def kinz_time(kinz_data):
    frames_num = len(kinz_data)
    time_in_sec = np.zeros(frames_num)
    time0 = kinz_data[0]  # time zero for the acquisition

    for i in range(frames_num):
        time_dbl = kinz_data[i] - time0  # relative time (double precision)
        time_in_sec[i] = time_dbl / 1e9  # Convert time to seconds.

    mean_time_diff = np.mean(np.diff(time_in_sec))  # Mean time difference
    fs = 1 / mean_time_diff  # Sampling rate

    return time_in_sec, fs

In [ ]:
from pymatreader import read_mat
import pandas as pd
import numpy as np
from glob import glob
from tqdm.notebook import tqdm

#data_path = '/home/iqh4001/Iqram_WCM/MADE Lab/Kinect Project/gait-analysis-main_VT/AI_Frailty-MATLAB/AI_Frailty-MATLAB'

data_path = "/content/drive/MyDrive/Iqram Sir/AI_Frailty-MATLAB_Sample"

In [ ]:
all_file_path=[]

for i in sorted(glob(data_path+"/*")):
  all_file_path = all_file_path+glob(i+"/*.mat")
  print(i)

/content/drive/MyDrive/Iqram Sir/AI_Frailty-MATLAB_Sample/001-LO


In [ ]:
all_file_path

[]

In [ ]:
import plotly.graph_objs as go

# Provided coordinates
coordinates = kinzData["kinzData"]['bodies'][i]["Orientation"]#['Position3d']

# Extracting x, y, z coordinates from the array
x_coords = coordinates[0]
y_coords = coordinates[1]
z_coords = coordinates[2]

# Create a 3D scatter plot
trace = go.Scatter3d(
    x=x_coords,
    y=y_coords,
    z=z_coords,
    mode='markers+lines',  # This will create both points and lines connecting them
    marker=dict(
        size=5,
        color='blue'  # The color of the points
    ),
    line=dict(
        color='blue',  # The color of the connecting lines
        width=2
    )
)

# Define layout for the 3D plot
layout = go.Layout(
    title='3D Coordinates Plot',
    scene=dict(
        xaxis=dict(title='X-axis'),
        yaxis=dict(title='Y-axis'),
        zaxis=dict(title='Z-axis')
    ),
    margin=dict(l=0, r=0, b=0, t=50)
)

# Create the figure and display it
fig = go.Figure(data=[trace], layout=layout)
fig.show()


In [ ]:
!pip install torch_geometric

In [ ]:
i

402

In [ ]:
import torch
import pandas as pd
import plotly.graph_objs as go
from torch_geometric.data import Data

# Define anatomical connections as edges
edges = torch.tensor([
    [0, 2],  # FOOT_RIGHT to ANKLE_RIGHT
    [1, 3],  # FOOT_LEFT to ANKLE_LEFT
    [2, 4],  # ANKLE_RIGHT to KNEE_RIGHT
    [3, 5],  # ANKLE_LEFT to KNEE_LEFT
    [4, 6],  # KNEE_RIGHT to HIP_RIGHT
    [5, 7],  # KNEE_LEFT to HIP_LEFT
    [6, 8],  # HIP_RIGHT to PELVIS
    [7, 8],  # HIP_LEFT to PELVIS
    [8, 9],  # PELVIS to SPINE_NAVAL
    [9, 10], # SPINE_NAVAL to SPINE_CHEST
    [10, 11],# SPINE_CHEST to CLAVICLE_RIGHT
    [10, 12],# SPINE_CHEST to CLAVICLE_LEFT
    [11, 13],# CLAVICLE_RIGHT to SHOULDER_RIGHT
    [12, 14],# CLAVICLE_LEFT to SHOULDER_LEFT
    [13, 15],# SHOULDER_RIGHT to ELBOW_RIGHT
    [14, 16],# SHOULDER_LEFT to ELBOW_LEFT
    [15, 17],# ELBOW_RIGHT to WRIST_RIGHT
    [16, 18],# ELBOW_LEFT to WRIST_LEFT
    [17, 19],# WRIST_RIGHT to HAND_RIGHT
    [18, 20],# WRIST_LEFT to HAND_LEFT
    [19, 21],# HAND_RIGHT to HANDTIP_RIGHT
    [20, 22],# HAND_LEFT to HANDTIP_LEFT
    [21, 23],# HANDTIP_RIGHT to THUMB_RIGHT
    [22, 24],# HANDTIP_LEFT to THUMB_LEFT
    [10, 25],# SPINE_CHEST to NECK
    [25, 26],# NECK to HEAD
]).t().contiguous()

# List of joints in the order they appear
joints = [
    'FOOT_RIGHT', 'FOOT_LEFT', 'ANKLE_RIGHT', 'ANKLE_LEFT', 'KNEE_RIGHT', 'KNEE_LEFT',
    'HIP_RIGHT', 'HIP_LEFT', 'PELVIS', 'SPINE_NAVAL', 'SPINE_CHEST',
    'CLAVICLE_RIGHT', 'CLAVICLE_LEFT', 'SHOULDER_RIGHT', 'SHOULDER_LEFT',
    'ELBOW_RIGHT', 'ELBOW_LEFT', 'WRIST_RIGHT', 'WRIST_LEFT', 'HAND_RIGHT',
    'HAND_LEFT', 'HANDTIP_RIGHT', 'HANDTIP_LEFT', 'THUMB_RIGHT', 'THUMB_LEFT',
    'NECK', 'HEAD'
]

# Function to create mock data from the given array
def create_mock_data_from_array():
    # Given array representing X, Y, Z coordinates
    mock_array = kinzData["kinzData"]['bodies'][402]['Position3d']
    # Convert the array to a DataFrame with appropriate labels
    mock_data = {}
    for i, joint in enumerate(joints):
        mock_data[f'{joint}_X'] = mock_array[0][i]
        mock_data[f'{joint}_Y'] = mock_array[1][i]
        mock_data[f'{joint}_Z'] = mock_array[2][i]

    return pd.Series(mock_data)

# Function to extract node features and create a Data object
def create_data_object_vis(row):
    node_features = []
    node_positions = []  # Store (x, y, z) positions separately for Plotly visualization

    for joint in joints:
        x = row[f'{joint}_X']
        y = row[f'{joint}_Y']
        z = row[f'{joint}_Z']
        node_features.append([x, y, z])
        node_positions.append((x, y, z))  # Save position for each joint

    node_features = torch.tensor(node_features, dtype=torch.float)

    # No label in this mock data
    label = torch.tensor([0], dtype=torch.float)

    # Create the Data object
    data = Data(x=node_features, edge_index=edges, y=label)

    return data, node_positions

# Function to plot the graph using Plotly
def plot_graph_large(node_positions):
    # Extract x, y, z coordinates from node_positions
    x_coords = [pos[0] for pos in node_positions]
    y_coords = [pos[1] for pos in node_positions]
    z_coords = [pos[2] for pos in node_positions]

    # Create edges for Plotly
    edge_x = []
    edge_y = []
    edge_z = []
    for edge in edges.t().tolist():
        x0, y0, z0 = node_positions[edge[0]]
        x1, y1, z1 = node_positions[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])
        edge_z.extend([z0, z1, None])

    # Plot the edges
    edge_trace = go.Scatter3d(
        x=edge_x, y=edge_y, z=edge_z,
        mode='lines',
        line=dict(color='black', width=3),  # Increased width for better visibility
        hoverinfo='none'
    )

    # Plot the nodes with joint names
    node_trace = go.Scatter3d(
        x=x_coords, y=y_coords, z=z_coords,
        mode='markers+text',
        marker=dict(size=8, color='blue'),  # Increased size for better visibility
        text=joints,  # Use the joint names as text labels
        hoverinfo='text'
    )

    # Create the figure with increased size
    fig = go.Figure(data=[edge_trace, node_trace],
                    layout=go.Layout(
                        title='3D Humanoid Skeleton Visualization',
                        showlegend=False,
                        scene=dict(
                            xaxis=dict(
                                showbackground=True,
                                backgroundcolor="rgb(230, 230, 230)",
                                gridcolor="rgb(200, 200, 200)",
                                showgrid=True,
                                zerolinecolor="rgb(200, 200, 200)",
                            ),
                            yaxis=dict(
                                showbackground=True,
                                backgroundcolor="rgb(230, 230, 230)",
                                gridcolor="rgb(200, 200, 200)",
                                showgrid=True,
                                zerolinecolor="rgb(200, 200, 200)",
                            ),
                            zaxis=dict(
                                showbackground=True,
                                backgroundcolor="rgb(230, 230, 230)",
                                gridcolor="rgb(200, 200, 200)",
                                showgrid=True,
                                zerolinecolor="rgb(200, 200, 200)",
                            ),
                        ),
                        width=1200,  # Increased width
                        height=800,  # Increased height
                        margin=dict(l=0, r=0, b=0, t=40),
                    ))

    fig.show()

# Create mock data from the given array
mock_data = create_mock_data_from_array()

# Convert the mock data to a Data object and node positions
data, node_positions = create_data_object_vis(mock_data)

# Plot the larger graph using Plotly
plot_graph_large(node_positions)


In [ ]:

import pandas as pd
import plotly.graph_objs as go
from torch_geometric.data import Data

# Define anatomical connections as edges based on the provided hierarchy
edges = torch.tensor([
    [0, 1],   # PELVIS to SPINE_NAVAL
    [1, 2],   # SPINE_NAVAL to SPINE_CHEST
    [2, 3],   # SPINE_CHEST to NECK
    [2, 4],   # SPINE_CHEST to CLAVICLE_LEFT
    [4, 5],   # CLAVICLE_LEFT to SHOULDER_LEFT
    [5, 6],   # SHOULDER_LEFT to ELBOW_LEFT
    [6, 7],   # ELBOW_LEFT to WRIST_LEFT
    [7, 8],   # WRIST_LEFT to HAND_LEFT
    [8, 9],   # HAND_LEFT to HANDTIP_LEFT
    [7, 10],  # WRIST_LEFT to THUMB_LEFT
    [2, 11],  # SPINE_CHEST to CLAVICLE_RIGHT
    [11, 12], # CLAVICLE_RIGHT import torch
    [12, 13], # SHOULDER_RIGHT to ELBOW_RIGHT
    [13, 14], # ELBOW_RIGHT to WRIST_RIGHT
    [14, 15], # WRIST_RIGHT to HAND_RIGHT
    [15, 16], # HAND_RIGHT to HANDTIP_RIGHT
    [14, 17], # WRIST_RIGHT to THUMB_RIGHT
    [0, 18],  # PELVIS to HIP_LEFT
    [18, 19], # HIP_LEFT to KNEE_LEFT
    # Add more edges as per the hierarchy if necessary
]).t().contiguous()

# List of joints in the order they appear based on the provided hierarchy
joints = [
    'PELVIS', 'SPINE_NAVAL', 'SPINE_CHEST', 'NECK', 'CLAVICLE_LEFT', 'SHOULDER_LEFT',
    'ELBOW_LEFT', 'WRIST_LEFT', 'HAND_LEFT', 'HANDTIP_LEFT', 'THUMB_LEFT',
    'CLAVICLE_RIGHT', 'SHOULDER_RIGHT', 'ELBOW_RIGHT', 'WRIST_RIGHT', 'HAND_RIGHT',
    'HANDTIP_RIGHT', 'THUMB_RIGHT', 'HIP_LEFT', 'KNEE_LEFT'
    # Add more joints if necessary
]

# Function to create mock data from the given array
def create_mock_data_from_array():
    # Given array representing X, Y, Z coordinates
    mock_array = kinzData["kinzData"]['bodies'][402]['Position3d']


    # Convert the array to a DataFrame with appropriate labels
    mock_data = {}
    for i, joint in enumerate(joints):
        mock_data[f'{joint}_X'] = mock_array[0][i]
        mock_data[f'{joint}_Y'] = mock_array[1][i]
        mock_data[f'{joint}_Z'] = mock_array[2][i]

    return pd.Series(mock_data)

# Function to extract node features and create a Data object
def create_data_object_vis(row):
    node_features = []
    node_positions = []  # Store (x, y, z) positions separately for Plotly visualization

    for joint in joints:
        x = row[f'{joint}_X']
        y = row[f'{joint}_Y']
        z = row[f'{joint}_Z']
        node_features.append([x, y, z])
        node_positions.append((x, y, z))  # Save position for each joint

    node_features = torch.tensor(node_features, dtype=torch.float)

    # No label in this mock data
    label = torch.tensor([0], dtype=torch.float)

    # Create the Data object
    data = Data(x=node_features, edge_index=edges, y=label)

    return data, node_positions

# Function to plot the graph using Plotly
def plot_graph_large(node_positions):
    # Extract x, y, z coordinates from node_positions
    x_coords = [pos[0] for pos in node_positions]
    y_coords = [pos[1] for pos in node_positions]
    z_coords = [pos[2] for pos in node_positions]

    # Create edges for Plotly
    edge_x = []
    edge_y = []
    edge_z = []
    for edge in edges.t().tolist():
        x0, y0, z0 = node_positions[edge[0]]
        x1, y1, z1 = node_positions[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])
        edge_z.extend([z0, z1, None])

    # Plot the edges
    edge_trace = go.Scatter3d(
        x=edge_x, y=edge_y, z=edge_z,
        mode='lines',
        line=dict(color='black', width=3),  # Increased width for better visibility
        hoverinfo='none'
    )

    # Plot the nodes with joint names
    node_trace = go.Scatter3d(
        x=x_coords, y=y_coords, z=z_coords,
        mode='markers+text',
        marker=dict(size=8, color='blue'),  # Increased size for better visibility
        text=joints,  # Use the joint names as text labels
        hoverinfo='text'
    )

    # Create the figure with increased size
    fig = go.Figure(data=[edge_trace, node_trace],
                    layout=go.Layout(
                        title='3D Humanoid Skeleton Visualization',
                        showlegend=False,
                        scene=dict(
                            xaxis=dict(
                                showbackground=True,
                                backgroundcolor="rgb(230, 230, 230)",
                                gridcolor="rgb(200, 200, 200)",
                                showgrid=True,
                                zerolinecolor="rgb(200, 200, 200)",
                            ),
                            yaxis=dict(
                                showbackground=True,
                                backgroundcolor="rgb(230, 230, 230)",
                                gridcolor="rgb(200, 200, 200)",
                                showgrid=True,
                                zerolinecolor="rgb(200, 200, 200)",
                            ),
                            zaxis=dict(
                                showbackground=True,
                                backgroundcolor="rgb(230, 230, 230)",
                                gridcolor="rgb(200, 200, 200)",
                                showgrid=True,
                                zerolinecolor="rgb(200, 200, 200)",
                            ),
                        ),
                        width=1200,  # Increased width
                        height=800,  # Increased height
                        margin=dict(l=0, r=0, b=0, t=40),
                    ))

    fig.show()

# Create mock data from the given array
mock_data = create_mock_data_from_array()

# Convert the mock data to a Data object and node positions
data, node_positions = create_data_object_vis(mock_data)

# Plot the larger graph using Plotly
plot_graph_large(node_positions)


In [ ]:
import torch
import pandas as pd
import plotly.graph_objs as go
from torch_geometric.data import Data

# Define anatomical connections as edges based on the provided hierarchy
edges = torch.tensor([
    [0, 1],   # PELVIS to SPINE_NAVAL
    [1, 2],   # SPINE_NAVAL to SPINE_CHEST
    [2, 3],   # SPINE_CHEST to NECK
    [2, 4],   # SPINE_CHEST to CLAVICLE_LEFT
    [4, 5],   # CLAVICLE_LEFT to SHOULDER_LEFT
    [5, 6],   # SHOULDER_LEFT to ELBOW_LEFT
    [6, 7],   # ELBOW_LEFT to WRIST_LEFT
    [7, 8],   # WRIST_LEFT to HAND_LEFT
    [8, 9],   # HAND_LEFT to HANDTIP_LEFT
    [7, 10],  # WRIST_LEFT to THUMB_LEFT
    [2, 11],  # SPINE_CHEST to CLAVICLE_RIGHT
    [11, 12], # CLAVICLE_RIGHT to SHOULDER_RIGHT
    [12, 13], # SHOULDER_RIGHT to ELBOW_RIGHT
    [13, 14], # ELBOW_RIGHT to WRIST_RIGHT
    [14, 15], # WRIST_RIGHT to HAND_RIGHT
    [15, 16], # HAND_RIGHT to HANDTIP_RIGHT
    [14, 17], # WRIST_RIGHT to THUMB_RIGHT
    [0, 18],  # PELVIS to HIP_LEFT
    [18, 19], # HIP_LEFT to KNEE_LEFT
    [19, 20], # KNEE_LEFT to ANKLE_LEFT
    [20, 21], # ANKLE_LEFT to FOOT_LEFT
    [0, 22],  # PELVIS to HIP_RIGHT
    [22, 23], # HIP_RIGHT to KNEE_RIGHT
    [23, 24], # KNEE_RIGHT to ANKLE_RIGHT
    [24, 25], # ANKLE_RIGHT to FOOT_RIGHT
]).t().contiguous()

# List of joints in the order they appear based on the provided hierarchy
joints = [
    'PELVIS', 'SPINE_NAVAL', 'SPINE_CHEST', 'NECK', 'CLAVICLE_LEFT', 'SHOULDER_LEFT',
    'ELBOW_LEFT', 'WRIST_LEFT', 'HAND_LEFT', 'HANDTIP_LEFT', 'THUMB_LEFT',
    'CLAVICLE_RIGHT', 'SHOULDER_RIGHT', 'ELBOW_RIGHT', 'WRIST_RIGHT', 'HAND_RIGHT',
    'HANDTIP_RIGHT', 'THUMB_RIGHT', 'HIP_LEFT', 'KNEE_LEFT', 'ANKLE_LEFT',
    'FOOT_LEFT', 'HIP_RIGHT', 'KNEE_RIGHT', 'ANKLE_RIGHT', 'FOOT_RIGHT'
]

# Function to create mock data from the given array
def create_mock_data_from_array():
    # Given array representing X, Y, Z coordinates
    mock_array =kinzData["kinzData"]['bodies'][req_index[0]]['Position3d']

    # Convert the array to a DataFrame with appropriate labels
    mock_data = {}
    for i, joint in enumerate(joints):
        mock_data[f'{joint}_X'] = mock_array[0][i]
        mock_data[f'{joint}_Y'] = mock_array[1][i]
        mock_data[f'{joint}_Z'] = mock_array[2][i]

    return pd.Series(mock_data)

# Function to extract node features and create a Data object
def create_data_object_vis(row):
    node_features = []
    node_positions = []  # Store (x, y, z) positions separately for Plotly visualization

    for joint in joints:
        x = row[f'{joint}_X']
        y = row[f'{joint}_Y']
        z = row[f'{joint}_Z']
        node_features.append([x, y, z])
        node_positions.append((x, y, z))  # Save position for each joint

    node_features = torch.tensor(node_features, dtype=torch.float)

    # No label in this mock data
    label = torch.tensor([0], dtype=torch.float)

    # Create the Data object
    data = Data(x=node_features, edge_index=edges, y=label)

    return data, node_positions

# Function to plot the graph using Plotly
def plot_graph_large(node_positions):
    # Extract x, y, z coordinates from node_positions
    x_coords = [pos[0] for pos in node_positions]
    y_coords = [pos[1] for pos in node_positions]
    z_coords = [pos[2] for pos in node_positions]

    # Create edges for Plotly
    edge_x = []
    edge_y = []
    edge_z = []
    for edge in edges.t().tolist():
        x0, y0, z0 = node_positions[edge[0]]
        x1, y1, z1 = node_positions[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])
        edge_z.extend([z0, z1, None])

    # Plot the edges
    edge_trace = go.Scatter3d(
        x=edge_x, y=edge_y, z=edge_z,
        mode='lines',
        line=dict(color='black', width=3),  # Increased width for better visibility
        hoverinfo='none'
    )

    # Plot the nodes with joint names
    node_trace = go.Scatter3d(
        x=x_coords, y=y_coords, z=z_coords,
        mode='markers+text',
        marker=dict(size=8, color='blue'),  # Increased size for better visibility
        text=joints,  # Use the joint names as text labels
        hoverinfo='text'
    )

    # Create the figure with increased size
    fig = go.Figure(data=[edge_trace, node_trace],
                    layout=go.Layout(
                        title='3D Humanoid Skeleton Visualization',
                        showlegend=False,
                        scene=dict(
                            xaxis=dict(
                                showbackground=True,
                                backgroundcolor="rgb(230, 230, 230)",
                                gridcolor="rgb(200, 200, 200)",
                                showgrid=True,
                                zerolinecolor="rgb(200, 200, 200)",
                            ),
                            yaxis=dict(
                                showbackground=True,
                                backgroundcolor="rgb(230, 230, 230)",
                                gridcolor="rgb(200, 200, 200)",
                                showgrid=True,
                                zerolinecolor="rgb(200, 200, 200)",
                            ),
                            zaxis=dict(
                                showbackground=True,
                                backgroundcolor="rgb(230, 230, 230)",
                                gridcolor="rgb(200, 200, 200)",
                                showgrid=True,
                                zerolinecolor="rgb(200, 200, 200)",
                            ),
                        ),
                        width=1200,  # Increased width
                        height=800,  # Increased height
                        margin=dict(l=0, r=0, b=0, t=40),
                    ))

    fig.show()

# Create mock data from the given array
mock_data = create_mock_data_from_array()

# Convert the mock data to a Data object and node positions
data, node_positions = create_data_object_vis(mock_data)

# Plot the larger graph using Plotly
plot_graph_large(node_positions)


In [ ]:
import torch
import pandas as pd
import plotly.graph_objs as go
from torch_geometric.data import Data

# Define anatomical connections as edges based on the provided hierarchy
edges = torch.tensor([
    [0, 1],   # PELVIS to SPINE_NAVAL
    [1, 2],   # SPINE_NAVAL to SPINE_CHEST
    [2, 3],   # SPINE_CHEST to NECK
    [2, 4],   # SPINE_CHEST to CLAVICLE_LEFT
    [4, 5],   # CLAVICLE_LEFT to SHOULDER_LEFT
    [5, 6],   # SHOULDER_LEFT to ELBOW_LEFT
    [6, 7],   # ELBOW_LEFT to WRIST_LEFT
    [7, 8],   # WRIST_LEFT to HAND_LEFT
    [8, 9],   # HAND_LEFT to HANDTIP_LEFT
    [7, 10],  # WRIST_LEFT to THUMB_LEFT
    [2, 11],  # SPINE_CHEST to CLAVICLE_RIGHT
    [11, 12], # CLAVICLE_RIGHT to SHOULDER_RIGHT
    [12, 13], # SHOULDER_RIGHT to ELBOW_RIGHT
    [13, 14], # ELBOW_RIGHT to WRIST_RIGHT
    [14, 15], # WRIST_RIGHT to HAND_RIGHT
    [15, 16], # HAND_RIGHT to HANDTIP_RIGHT
    [14, 17], # WRIST_RIGHT to THUMB_RIGHT
    [0, 18],  # PELVIS to HIP_LEFT
    [18, 19], # HIP_LEFT to KNEE_LEFT
    [19, 20], # KNEE_LEFT to ANKLE_LEFT
    [20, 21], # ANKLE_LEFT to FOOT_LEFT
    [0, 22],  # PELVIS to HIP_RIGHT
    [22, 23], # HIP_RIGHT to KNEE_RIGHT
    [23, 24], # KNEE_RIGHT to ANKLE_RIGHT
    [24, 25], # ANKLE_RIGHT to FOOT_RIGHT
]).t().contiguous()

# List of joints in the order they appear based on the provided hierarchy
joints = [
    'PELVIS', 'SPINE_NAVAL', 'SPINE_CHEST', 'NECK', 'CLAVICLE_LEFT', 'SHOULDER_LEFT',
    'ELBOW_LEFT', 'WRIST_LEFT', 'HAND_LEFT', 'HANDTIP_LEFT', 'THUMB_LEFT',
    'CLAVICLE_RIGHT', 'SHOULDER_RIGHT', 'ELBOW_RIGHT', 'WRIST_RIGHT', 'HAND_RIGHT',
    'HANDTIP_RIGHT', 'THUMB_RIGHT', 'HIP_LEFT', 'KNEE_LEFT', 'ANKLE_LEFT',
    'FOOT_LEFT', 'HIP_RIGHT', 'KNEE_RIGHT', 'ANKLE_RIGHT', 'FOOT_RIGHT'
]

# Function to create mock data from the given array
def create_mock_data_from_array():
    # Given array representing X, Y, Z coordinates
    mock_array =kinzData["kinzData"]['bodies'][req_index[-1]]['Position3d']

    # Convert the array to a DataFrame with appropriate labels
    mock_data = {}
    for i, joint in enumerate(joints):
        mock_data[f'{joint}_X'] = mock_array[0][i]
        mock_data[f'{joint}_Y'] = mock_array[1][i]
        mock_data[f'{joint}_Z'] = mock_array[2][i]

    return pd.Series(mock_data)

# Function to extract node features and create a Data object
def create_data_object_vis(row):
    node_features = []
    node_positions = []  # Store (x, y, z) positions separately for Plotly visualization

    for joint in joints:
        x = row[f'{joint}_X']
        y = row[f'{joint}_Y']
        z = row[f'{joint}_Z']
        node_features.append([x, y, z])
        node_positions.append((x, y, z))  # Save position for each joint

    node_features = torch.tensor(node_features, dtype=torch.float)

    # No label in this mock data
    label = torch.tensor([0], dtype=torch.float)

    # Create the Data object
    data = Data(x=node_features, edge_index=edges, y=label)

    return data, node_positions

# Function to plot the graph using Plotly
def plot_graph_large(node_positions):
    # Extract x, y, z coordinates from node_positions
    x_coords = [pos[0] for pos in node_positions]
    y_coords = [pos[1] for pos in node_positions]
    z_coords = [pos[2] for pos in node_positions]

    # Create edges for Plotly
    edge_x = []
    edge_y = []
    edge_z = []
    for edge in edges.t().tolist():
        x0, y0, z0 = node_positions[edge[0]]
        x1, y1, z1 = node_positions[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])
        edge_z.extend([z0, z1, None])

    # Plot the edges
    edge_trace = go.Scatter3d(
        x=edge_x, y=edge_y, z=edge_z,
        mode='lines',
        line=dict(color='black', width=3),  # Increased width for better visibility
        hoverinfo='none'
    )

    # Plot the nodes with joint names
    node_trace = go.Scatter3d(
        x=x_coords, y=y_coords, z=z_coords,
        mode='markers+text',
        marker=dict(size=8, color='blue'),  # Increased size for better visibility
        text=joints,  # Use the joint names as text labels
        hoverinfo='text'
    )

    # Create the figure with increased size
    fig = go.Figure(data=[edge_trace, node_trace],
                    layout=go.Layout(
                        title='3D Humanoid Skeleton Visualization',
                        showlegend=False,
                        scene=dict(
                            xaxis=dict(
                                showbackground=True,
                                backgroundcolor="rgb(230, 230, 230)",
                                gridcolor="rgb(200, 200, 200)",
                                showgrid=True,
                                zerolinecolor="rgb(200, 200, 200)",
                            ),
                            yaxis=dict(
                                showbackground=True,
                                backgroundcolor="rgb(230, 230, 230)",
                                gridcolor="rgb(200, 200, 200)",
                                showgrid=True,
                                zerolinecolor="rgb(200, 200, 200)",
                            ),
                            zaxis=dict(
                                showbackground=True,
                                backgroundcolor="rgb(230, 230, 230)",
                                gridcolor="rgb(200, 200, 200)",
                                showgrid=True,
                                zerolinecolor="rgb(200, 200, 200)",
                            ),
                        ),
                        width=1200,  # Increased width
                        height=800,  # Increased height
                        margin=dict(l=0, r=0, b=0, t=40),
                    ))

    fig.show()

# Create mock data from the given array
mock_data = create_mock_data_from_array()

# Convert the mock data to a Data object and node positions
data, node_positions = create_data_object_vis(mock_data)

# Plot the larger graph using Plotly
plot_graph_large(node_positions)


In [ ]:
import os
from datetime import datetime

current_datetime = datetime.now()
formatted_datetime = current_datetime.strftime("%Y-%m-%d_%H-%M%p")

#file_save_destination = '/home/iqh4001/Iqram_WCM/MADE Lab/Kinect Project/Output Data/Kinect Data/kinzData_panda_'+formatted_datetime+'.csv'

file_save_destination = "task1.csv"


if os.path.exists(file_save_destination)==False:
  pd.DataFrame(np.empty([0,100]), columns=['patientID','trial_num','walking_speed','timeStamps',
  'PELVIS_X', 'SPINE_NAVAL_X', 'SPINE_CHEST_X', 'NECK_X', 'CLAVICLE_LEFT_X', 'SHOULDER_LEFT_X', 'ELBOW_LEFT_X', 'WRIST_LEFT_X', 'HAND_LEFT_X', 'HANDTIP_LEFT_X', 'THUMB_LEFT_X', 'CLAVICLE_RIGHT_X', 'SHOULDER_RIGHT_X', 'ELBOW_RIGHT_X', 'WRIST_RIGHT_X', 'HAND_RIGHT_X', 'HANDTIP_RIGHT_X', 'THUMB_RIGHT_X', 'HIP_LEFT_X', 'KNEE_LEFT_X', 'ANKLE_LEFT_X', 'FOOT_LEFT_X', 'HIP_RIGHT_X', 'KNEE_RIGHT_X', 'ANKLE_RIGHT_X', 'FOOT_RIGHT_X', 'HEAD_X', 'NOSE_X', 'EYE_LEFT_X', 'EAR_LEFT_X', 'EYE_RIGHT_X', 'EAR_RIGHT_X',
  'PELVIS_Y', 'SPINE_NAVAL_Y', 'SPINE_CHEST_Y', 'NECK_Y', 'CLAVICLE_LEFT_Y', 'SHOULDER_LEFT_Y', 'ELBOW_LEFT_Y', 'WRIST_LEFT_Y', 'HAND_LEFT_Y', 'HANDTIP_LEFT_Y', 'THUMB_LEFT_Y', 'CLAVICLE_RIGHT_Y', 'SHOULDER_RIGHT_Y', 'ELBOW_RIGHT_Y', 'WRIST_RIGHT_Y', 'HAND_RIGHT_Y', 'HANDTIP_RIGHT_Y', 'THUMB_RIGHT_Y', 'HIP_LEFT_Y', 'KNEE_LEFT_Y', 'ANKLE_LEFT_Y', 'FOOT_LEFT_Y', 'HIP_RIGHT_Y', 'KNEE_RIGHT_Y', 'ANKLE_RIGHT_Y', 'FOOT_RIGHT_Y', 'HEAD_Y', 'NOSE_Y', 'EYE_LEFT_Y', 'EAR_LEFT_Y', 'EYE_RIGHT_Y', 'EAR_RIGHT_Y',
  'PELVIS_Z', 'SPINE_NAVAL_Z', 'SPINE_CHEST_Z', 'NECK_Z', 'CLAVICLE_LEFT_Z', 'SHOULDER_LEFT_Z', 'ELBOW_LEFT_Z', 'WRIST_LEFT_Z', 'HAND_LEFT_Z', 'HANDTIP_LEFT_Z', 'THUMB_LEFT_Z', 'CLAVICLE_RIGHT_Z', 'SHOULDER_RIGHT_Z', 'ELBOW_RIGHT_Z', 'WRIST_RIGHT_Z', 'HAND_RIGHT_Z', 'HANDTIP_RIGHT_Z', 'THUMB_RIGHT_Z', 'HIP_LEFT_Z', 'KNEE_LEFT_Z', 'ANKLE_LEFT_Z', 'FOOT_LEFT_Z', 'HIP_RIGHT_Z', 'KNEE_RIGHT_Z', 'ANKLE_RIGHT_Z', 'FOOT_RIGHT_Z', 'HEAD_Z', 'NOSE_Z', 'EYE_LEFT_Z', 'EAR_LEFT_Z', 'EYE_RIGHT_Z', 'EAR_RIGHT_Z'
                                                ]).to_csv(file_save_destination,index=False)



try:

  for path in tqdm(all_file_path):
    data = read_mat(path)
    temp_df = pd.DataFrame(kinz_data(data), columns=['patientID','trial_num','walking_speed','timeStamps',
    'PELVIS_X', 'SPINE_NAVAL_X', 'SPINE_CHEST_X', 'NECK_X', 'CLAVICLE_LEFT_X', 'SHOULDER_LEFT_X', 'ELBOW_LEFT_X', 'WRIST_LEFT_X', 'HAND_LEFT_X', 'HANDTIP_LEFT_X', 'THUMB_LEFT_X', 'CLAVICLE_RIGHT_X', 'SHOULDER_RIGHT_X', 'ELBOW_RIGHT_X', 'WRIST_RIGHT_X', 'HAND_RIGHT_X', 'HANDTIP_RIGHT_X', 'THUMB_RIGHT_X', 'HIP_LEFT_X', 'KNEE_LEFT_X', 'ANKLE_LEFT_X', 'FOOT_LEFT_X', 'HIP_RIGHT_X', 'KNEE_RIGHT_X', 'ANKLE_RIGHT_X', 'FOOT_RIGHT_X', 'HEAD_X', 'NOSE_X', 'EYE_LEFT_X', 'EAR_LEFT_X', 'EYE_RIGHT_X', 'EAR_RIGHT_X',
    'PELVIS_Y', 'SPINE_NAVAL_Y', 'SPINE_CHEST_Y', 'NECK_Y', 'CLAVICLE_LEFT_Y', 'SHOULDER_LEFT_Y', 'ELBOW_LEFT_Y', 'WRIST_LEFT_Y', 'HAND_LEFT_Y', 'HANDTIP_LEFT_Y', 'THUMB_LEFT_Y', 'CLAVICLE_RIGHT_Y', 'SHOULDER_RIGHT_Y', 'ELBOW_RIGHT_Y', 'WRIST_RIGHT_Y', 'HAND_RIGHT_Y', 'HANDTIP_RIGHT_Y', 'THUMB_RIGHT_Y', 'HIP_LEFT_Y', 'KNEE_LEFT_Y', 'ANKLE_LEFT_Y', 'FOOT_LEFT_Y', 'HIP_RIGHT_Y', 'KNEE_RIGHT_Y', 'ANKLE_RIGHT_Y', 'FOOT_RIGHT_Y', 'HEAD_Y', 'NOSE_Y', 'EYE_LEFT_Y', 'EAR_LEFT_Y', 'EYE_RIGHT_Y', 'EAR_RIGHT_Y',
    'PELVIS_Z', 'SPINE_NAVAL_Z', 'SPINE_CHEST_Z', 'NECK_Z', 'CLAVICLE_LEFT_Z', 'SHOULDER_LEFT_Z', 'ELBOW_LEFT_Z', 'WRIST_LEFT_Z', 'HAND_LEFT_Z', 'HANDTIP_LEFT_Z', 'THUMB_LEFT_Z', 'CLAVICLE_RIGHT_Z', 'SHOULDER_RIGHT_Z', 'ELBOW_RIGHT_Z', 'WRIST_RIGHT_Z', 'HAND_RIGHT_Z', 'HANDTIP_RIGHT_Z', 'THUMB_RIGHT_Z', 'HIP_LEFT_Z', 'KNEE_LEFT_Z', 'ANKLE_LEFT_Z', 'FOOT_LEFT_Z', 'HIP_RIGHT_Z', 'KNEE_RIGHT_Z', 'ANKLE_RIGHT_Z', 'FOOT_RIGHT_Z', 'HEAD_Z', 'NOSE_Z', 'EYE_LEFT_Z', 'EAR_LEFT_Z', 'EYE_RIGHT_Z', 'EAR_RIGHT_Z'
                                                  ])

    columns_to_convert = temp_data.columns.difference(['patientID','trial_num', 'walking_speed'])
    temp_data[columns_to_convert] = temp_data[columns_to_convert].astype(float)

    k = kinz_time(temp_data["timeStamps"])
    temp_data["timeStamps"] = k[0]
    temp_df.to_csv(file_save_destination, mode='a', index=False, header=False)


except:
  print("The Subject folder or main path unusual file or folder below.....Please remove the file or the folder.....")
  print(path)

  0%|          | 0/24 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/pymatreader/utils.py:238: UserWarning: Complex objects (like classes) are not supported. They are imported on a best effort base but your mileage will vary.
  warn('Complex objects (like classes) are not supported. '
/usr/local/lib/python3.10/dist-packages/pymatreader/utils.py:238: UserWarning: Complex objects (like classes) are not supported. They are imported on a best effort base but your mileage will vary.
  warn('Complex objects (like classes) are not supported. '
/usr/local/lib/python3.10/dist-packages/pymatreader/utils.py:238: UserWarning: Complex objects (like classes) are not supported. They are imported on a best effort base but your mileage will vary.
  warn('Complex objects (like classes) are not supported. '
/usr/local/lib/python3.10/dist-packages/pymatreader/utils.py:238: UserWarning: Complex objects (like classes) are not supported. They are imported on a best effort base but your mileage will vary.
  warn('Complex objects (like cl

In [ ]:
import os
from datetime import datetime

current_datetime = datetime.now()
formatted_datetime = current_datetime.strftime("%Y-%m-%d_%H-%M%p")

#file_save_destination_1 = '/home/iqh4001/Iqram_WCM/MADE Lab/Kinect Project/Output Data/ACC Data/ACC_Data_panda_'+formatted_datetime+'.csv'

file_save_destination_1 = "task2.csv"

if os.path.exists(file_save_destination_1)==False:
  pd.DataFrame(np.empty([0,12]), columns=['patientID','trial_num','walking_speed','stopwatch','timeStamps','acc_1', 'acc_2', 'acc_3','acc_4', 'acc_5', 'acc_6','acc_7']).to_csv(file_save_destination_1,index=False)




for path in tqdm(all_file_path):
  data = read_mat(path)
  temp_df = pd.DataFrame(acc_data(data), columns=['patientID','trial_num','walking_speed','stopwatch','timeStamps','acc_1', 'acc_2', 'acc_3',
                                  'acc_4', 'acc_5', 'acc_6','acc_7'])


  columns_to_convert = temp_data.columns.difference(['patientID','trial_num','walking_speed','stopwatch'])
  temp_data[columns_to_convert] = temp_data[columns_to_convert].astype(float)

  k = kinz_time(temp_data["timeStamps"])
  temp_data["timeStamps"] = k[0]

  temp_df.to_csv(file_save_destination_1, mode='a', index=False, header=False)



  0%|          | 0/24 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/pymatreader/utils.py:238: UserWarning: Complex objects (like classes) are not supported. They are imported on a best effort base but your mileage will vary.
  warn('Complex objects (like classes) are not supported. '
/usr/local/lib/python3.10/dist-packages/pymatreader/utils.py:238: UserWarning: Complex objects (like classes) are not supported. They are imported on a best effort base but your mileage will vary.
  warn('Complex objects (like classes) are not supported. '
/usr/local/lib/python3.10/dist-packages/pymatreader/utils.py:238: UserWarning: Complex objects (like classes) are not supported. They are imported on a best effort base but your mileage will vary.
  warn('Complex objects (like classes) are not supported. '
/usr/local/lib/python3.10/dist-packages/pymatreader/utils.py:238: UserWarning: Complex objects (like classes) are not supported. They are imported on a best effort base but your mileage will vary.
  warn('Complex objects (like cl